In [2]:
import pandas as pd
import numpy as np
import graphviz


In [9]:
def calculate_twoing_criterion(data, target_column):
    """
    Calculates the twoing criterion for splits within each column individually.
    """
    best_split_combination = None
    best_criterion_value = -1

    split_columns = [col for col in data.columns if col != target_column]

    for col in split_columns:
        for split_value in data[col].unique():
            data_copy = data.copy()
            data_copy[col] = data_copy[col].apply(lambda x: split_value if x == split_value else 'Other')

            group_counts = data_copy.groupby(col)[target_column].value_counts()
            probabilities = group_counts / group_counts.groupby(level=0).sum()
            probability_table = probabilities.unstack(fill_value=0)

            if len(probability_table.columns) == 1:
                probability_table['dummy'] = 0

            probability_table.columns = ['P(No)', 'P(Yes)'][:len(probability_table.columns)]
            group_ratios = data_copy[col].value_counts() / len(data_copy)
            probability_table['Group Ratio'] = group_ratios

            try:
                normal_yes_prob = probability_table.loc[split_value, 'P(Yes)']
                diger_yes_prob = probability_table.loc['Other', 'P(Yes)']
                normal_no_prob = probability_table.loc[split_value, 'P(No)']
                diger_no_prob = probability_table.loc['Other', 'P(No)']
            except KeyError:
                continue

            total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)
            criterion_value = (2 * total_diff * group_ratios[split_value] * group_ratios['Other'])

            if criterion_value > best_criterion_value:
                best_criterion_value = criterion_value
                best_split_combination = (col, split_value)

    return best_split_combination, best_criterion_value

In [10]:
def build_tree(data, target_column):
    """
    Builds a decision tree recursively using the twoing criterion.
    """
    if len(data[target_column].unique()) == 1:
        return data[target_column].iloc[0]

    best_split_combination, best_criterion_value = calculate_twoing_criterion(data, target_column)

    if best_split_combination is None:
        return data[target_column].value_counts().idxmax()

    split_column = best_split_combination[0]
    split_value = best_split_combination[1]

    left_split = data[data[split_column] == split_value]
    right_split = data[data[split_column] != split_value]

    if len(left_split) == 0 or len(right_split) == 0:
        return data[target_column].value_counts().idxmax()

    left_leaf = build_tree(left_split, target_column)
    right_leaf = build_tree(right_split, target_column)

    tree = {split_column: {split_value: left_leaf, 'Other': right_leaf}}
    return tree

In [11]:
def visualize_tree(tree):
    """
    Recursively visualizes the decision tree with labeled branches.
    """
    dot = graphviz.Digraph(comment="Decision Tree")

    def add_nodes_edges(subtree, parent_name, parent_label=""):
        for key, value in subtree.items():
            # Düğümün etiketi
            edge_label = str(key)

            if isinstance(value, dict):
                child_name = f"{parent_name}_{key}"
                dot.node(child_name, label=key)
                dot.edge(parent_name, child_name, label=edge_label)
                add_nodes_edges(value, child_name)
            else:
                leaf_name = f"{parent_name}_{key}_leaf"
                dot.node(leaf_name, label=str(value), shape="ellipse")
                dot.edge(parent_name, leaf_name, label=edge_label)

    root_name = "root"
    dot.node(root_name, label="root")
    add_nodes_edges(tree, root_name)

    return dot

In [14]:
def print_tree_rules(tree, indent=""):
    """
    Karar ağacındaki kuralları yazdıran fonksiyon.
    tree: Karar ağacı (ağaç yapısının kökü)
    indent: İç içe geçmiş düğümleri doğru şekilde yazdırmak için boşluklar
    """
    if isinstance(tree, dict):  # Düğümde bölünme varsa
        for feature, sub_tree in tree.items():
            # 'feature' (özellik) üzerinde karar yapılmış
            for value, subtree in sub_tree.items():
                print(f"{indent}If {feature} = {value}:")
                print_tree_rules(subtree, indent + "  ")
    else:  # Bir yaprak (sonuç sınıfı) ile karşılaşıldığında
        print(f"{indent}Result: {tree}")



In [15]:

file_path = input("CSV dosyasının yolunu girin: ")  # Kullanıcı dosya yolunu girmeli
target_column = input("Hedef sütun adını girin: ")  # Kullanıcı hedef sütunu girmeli
output_image_name = input("Görsel dosya ismi (uzantısız) girin: ")  # Kullanıcı resim dosya adını girmeli

# CSV'yi okuyun
data = pd.read_csv(file_path)

# Karar ağacını oluşturun
tree = build_tree(data, target_column)
print("Karar ağacı:", tree)

# Karar ağacını görselleştirin
decision_tree_graph = visualize_tree(tree)
decision_tree_graph.render(output_image_name, format="png", view=True)  # PNG olarak kaydedilir ve açılır
# Karar ağacını yazdırmak ve kuralları görmek için
print("Decision Tree Rules:")
print_tree_rules(tree)

CSV dosyasının yolunu girin: /content/twoing_algorithms.csv
Hedef sütun adını girin: Satisfaction
Görsel dosya ismi (uzantısız) girin: Tree
Karar ağacı: {'Sector': {'IT': {'Income': {'Normal': 'Yes', 'Other': {'Education': {'Primary': 'Yes', 'Other': 'No'}}}}, 'Other': 'Yes'}}
Decision Tree Rules:
If Sector = IT:
  If Income = Normal:
    Result: Yes
  If Income = Other:
    If Education = Primary:
      Result: Yes
    If Education = Other:
      Result: No
If Sector = Other:
  Result: Yes


# Full Hali

In [7]:
import pandas as pd
import graphviz

def calculate_twoing_criterion(data, target_column):
    """
    Calculates the twoing criterion for splits within each column individually.
    """
    best_split_combination = None
    best_criterion_value = -1

    split_columns = [col for col in data.columns if col != target_column]

    for col in split_columns:
        for split_value in data[col].unique():
            data_copy = data.copy()
            data_copy[col] = data_copy[col].apply(lambda x: split_value if x == split_value else 'Diğer')

            group_counts = data_copy.groupby(col)[target_column].value_counts()
            probabilities = group_counts / group_counts.groupby(level=0).sum()
            probability_table = probabilities.unstack(fill_value=0)

            if len(probability_table.columns) == 1:
                probability_table['dummy'] = 0

            probability_table.columns = ['P(No)', 'P(Yes)'][:len(probability_table.columns)]
            group_ratios = data_copy[col].value_counts() / len(data_copy)
            probability_table['Group Ratio'] = group_ratios

            try:
                normal_yes_prob = probability_table.loc[split_value, 'P(Yes)']
                diger_yes_prob = probability_table.loc['Other', 'P(Yes)']
                normal_no_prob = probability_table.loc[split_value, 'P(No)']
                diger_no_prob = probability_table.loc['Other', 'P(No)']
            except KeyError:
                continue

            total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)
            criterion_value = (2 * total_diff * group_ratios[split_value] * group_ratios['Other'])

            if criterion_value > best_criterion_value:
                best_criterion_value = criterion_value
                best_split_combination = (col, split_value)

    return best_split_combination, best_criterion_value

def build_tree(data, target_column):
    """
    Builds a decision tree recursively using the twoing criterion.
    """
    if len(data[target_column].unique()) == 1:
        return data[target_column].iloc[0]

    best_split_combination, best_criterion_value = calculate_twoing_criterion(data, target_column)

    if best_split_combination is None:
        return data[target_column].value_counts().idxmax()

    split_column = best_split_combination[0]
    split_value = best_split_combination[1]

    left_split = data[data[split_column] == split_value]
    right_split = data[data[split_column] != split_value]

    if len(left_split) == 0 or len(right_split) == 0:
        return data[target_column].value_counts().idxmax()

    left_leaf = build_tree(left_split, target_column)
    right_leaf = build_tree(right_split, target_column)

    tree = {split_column: {split_value: left_leaf, 'Other': right_leaf}}
    return tree

def visualize_tree(tree):
    """
    Recursively visualizes the decision tree with labeled branches.
    """
    dot = graphviz.Digraph(comment="Decision Tree")

    def add_nodes_edges(subtree, parent_name, parent_label=""):
        for key, value in subtree.items():
            # Düğümün etiketi
            edge_label = str(key)

            if isinstance(value, dict):
                child_name = f"{parent_name}_{key}"
                dot.node(child_name, label=key)
                dot.edge(parent_name, child_name, label=edge_label)
                add_nodes_edges(value, child_name)
            else:
                leaf_name = f"{parent_name}_{key}_leaf"
                dot.node(leaf_name, label=str(value), shape="ellipse")
                dot.edge(parent_name, leaf_name, label=edge_label)

    root_name = "root"
    dot.node(root_name, label="root")
    add_nodes_edges(tree, root_name)

    return dot


file_path = input("CSV dosyasının yolunu girin: ")  # Kullanıcı dosya yolunu girmeli
target_column = input("Hedef sütun adını girin: ")  # Kullanıcı hedef sütunu girmeli
output_image_name = input("Görsel dosya ismi (uzantısız) girin: ")  # Kullanıcı resim dosya adını girmeli

# CSV'yi okuyun
data = pd.read_csv(file_path)

# Karar ağacını oluşturun
tree = build_tree(data, target_column)
print("Karar ağacı:", tree)

# Karar ağacını görselleştirin
decision_tree_graph = visualize_tree(tree)
decision_tree_graph.render(output_image_name, format="png", view=True)  # PNG olarak kaydedilir ve açılır

CSV dosyasının yolunu girin: /content/twoing_algorithms.csv
Hedef sütun adını girin: Satisfaction
Görsel dosya ismi (uzantısız) girin: tree1
Karar ağacı: {'Sector': {'IT': {'Income': {'Normal': 'Yes', 'Diğer': {'Education': {'Primary': 'Yes', 'Diğer': 'No'}}}}, 'Diğer': 'Yes'}}


'tree1.png'